In [1]:
#@title Install Dependencies
%%capture
!pip install transformers==3.1.0

In [2]:
from glob import glob
from tqdm.notebook import tqdm
from google.colab import drive
import os
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from itertools import cycle
cycol = cycle('bgrcmk')
drive.mount('/content/drive')
root_path='/content/drive/My Drive/interactions_data/aws_transcribe_test_data.pkl'

Mounted at /content/drive


In [3]:
interactions_calls=pd.read_pickle(root_path)

In [4]:
from transformers import pipeline
classifier = pipeline('zero-shot-classification',
                      model='joeddav/xlm-roberta-large-xnli')
LABELS='labels'
SCORES='scores'

In [5]:
labels=[ "positivo", "negativo", "neutro"]
res=classifier(['no estoy del lado de ningun bando'], labels,multi_class=False)
dict(zip(res[LABELS], res[SCORES]))

{'negativo': 0.005789743270725012,
 'neutro': 0.9753322601318359,
 'positivo': 0.018877986818552017}

In [6]:
interactions_calls.loc[:,'transcript']=interactions_calls.transcript.apply(lambda t: list(filter(None, t)) if t is not None else [])

In [7]:
interactions_calls

,user_id,rfc,id_contact,date_created,interaction_origin,des_description,des_template,interaction_type,ctr_medialocation,transcript,transcripts_aws,transcript_agent,transcript_client,agent_sent,client_sent
0,224784,SAPE880222EP5,NaN,2020-09-21 22:34:39,Agente,"Hola Edgar, ¿ como te puedo ayudar?",NaN,whatsapp,NaN,[],NaN,NaN,NaN,NaN,NaN
1,224784,SAPE880222EP5,NaN,2020-09-21 17:39:43,Cliente,hola,NaN,whatsapp,NaN,[],NaN,NaN,NaN,NaN,NaN
2,224784,SAPE880222EP5,NaN,2020-05-18 14:57:58,Agente,"Cualquier información adicional que requieras,...",NaN,whatsapp,NaN,[],NaN,NaN,NaN,NaN,NaN
3,224784,SAPE880222EP5,NaN,2020-05-12 21:04:22,Agente,"Hola, solicitamos nuevamente te contacte",NaN,whatsapp,NaN,[],NaN,NaN,NaN,NaN,NaN
4,224784,SAPE880222EP5,NaN,2020-05-11 20:36:17,Cliente,te lo mensione en la aprte de arriba,NaN,whatsapp,NaN,[],NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,1151225,MAMC760324DN4,NaN,2020-02-12 18:21:27,Sistema,NaN,agradecimiento_pago_puntual,whatsapp,NaN,[],NaN,NaN,NaN,NaN,NaN
457,273858,MAMC760324DN4,44450569-1a38-426e-a64a-2965178f4840,2020-08-04 21:55:22,OUTBOUND,Se intenta contactar al TT sin éxito.,NaN,Call,"""connect-4ea90fb20d17/connect/connect-konfio/C...",[grabe su mensaje despues del tono],{'jobName': '44450569-1a38-426e-a64a-2965178f4...,,grabe su mensaje después del tono .,[],[grabe su mensaje después del tono .]
458,1035889,SASE701107VBA,NaN,2020-09-02 15:00:34,Sistema,NaN,recordatorio_pago_domiciliado_002,whatsapp,NaN,[],NaN,NaN,NaN,NaN,NaN
459,1213280,SASE701107VBA,NaN,2020-08-31 18:18:59,Agente,"Entiendo, revise tu caso y la oferta ya no est...",NaN,whatsapp,NaN,[],NaN,NaN,NaN,NaN,NaN


In [8]:
def is_nan(x):
    return (x != x)
def predict(row,labels=labels,classifier=classifier,multi_class=False):
    sentiments=[]
    if is_nan(row):
      #vad_n
      return None
    elif len(row)<1:
      return None
    else:
      #prob_speech
      for sentence in row:
        #transcript
        res=classifier(sentence, labels,multi_class=multi_class)
        sentiments.append(dict(zip(res[LABELS], res[SCORES])))
    return {k: [dic[k] for dic in sentiments] for k in sentiments[0]}


In [9]:
from multiprocessing import Pool
from functools import partial 
predict_binary=partial(predict,labels=['positivo','negativo'],multi_class=True)
transcripts=interactions_calls['transcript'].values 
with Pool(8) as p:
  interactions_calls['sentiment']=[sentiments for sentiments in tqdm(p.imap(predict,transcripts),total=len(transcripts))]
with Pool(8) as p:
  interactions_calls['sentiment_binary']=[sentiments for sentiments in tqdm(p.imap(predict_binary,transcripts),total=len(transcripts))]

In [10]:
samples=interactions_calls[interactions_calls.sentiment.notnull()][['transcript','sentiment','sentiment_binary']]

In [11]:
samples

,transcript,sentiment,sentiment_binary
42,"[Sí buenas tardes, habló a Eugenio, Mira por a...","{'neutro': [0.3602646291255951, 0.297514855861...","{'positivo': [0.9380167126655579, 0.8379161357..."
60,"[la gripa, bueno, disculpe Es que siempre anda...","{'negativo': [0.5728645324707031, 0.0027203352...","{'negativo': [0.8840526938438416, 0.0011707491..."
61,"[la gripa, bueno, disculpe Es que siempre anda...","{'negativo': [0.5728645324707031, 0.0027203352...","{'negativo': [0.8840526938438416, 0.0011707491..."
100,"[Hola que tal Buenos días, Fabiola Barbosa, gr...","{'neutro': [0.3625732660293579, 0.359371036291...","{'positivo': [0.8919315338134766, 0.8835017085..."
101,"[Hola señorita buenas, bretón, Quiero ver cuán...","{'positivo': [0.8379055857658386, 0.3165355026...","{'positivo': [0.9803280830383301, 0.7234244942..."
118,[sí que le llama No se encuentra por el moment...,"{'positivo': [0.42626598477363586, 0.411001294...","{'positivo': [0.8024404048919678, 0.7981640100..."
158,"[buenas tardes, con, Eugenio Sánchez Salas, po...","{'neutro': [0.3880135416984558, 0.653856933116...","{'positivo': [0.8666232228279114, 0.2620355188..."
159,"[con Cristian Madrid, necesito hablar a ver la...","{'positivo': [0.48635050654411316, 0.370454937...","{'positivo': [0.926856517791748, 0.74032407999..."
160,"[Real Madrid unas caras, de parte de quién, de...","{'negativo': [0.5284739136695862, 0.2900031507...","{'negativo': [0.9681347608566284, 0.7734022736..."
177,"[Christian Madrid, Mira a ver si me explico ha...","{'positivo': [0.42064526677131653, 0.544958710...","{'positivo': [0.8863071799278259, 0.7881762981..."


In [12]:
import plotly.graph_objects as go
def plot_lines(sample):
  fig = go.Figure()
  for key in sample.keys():
    fig.add_trace(go.Scatter(x=list(range(len(sample[key]))), y=sample[key], fill='tozeroy',name=key)) # fill down to xaxi
  fig.show()

In [13]:
for sample in samples.to_dict('records'):
  print(sample['transcript'])
  plot_lines(sample['sentiment'])
  plot_lines(sample['sentiment_binary'])

['Sí buenas tardes', 'habló a Eugenio', 'Mira por aquí me interesa estoy interesado en una reestructuración', 'multiservicios punto azul o arroba hotmail.com', 'Eugenio Sánchez Salas', 'el 7 de noviembre del 70 fecha de nacimiento el 7 de noviembre 70', 'correcto', 'Sí está bien', 'Bueno si estoy bien ya estamos ahí', 'Ian holm', 'Joan hart', 'John Holt', 'John Holt', 'arco el correo multiservicios multi el mismo correo multiservicios punto solo', 'un correo que lo tengo que compartir', 'Perdón no no le no le no le no le entiendo O sea qué es lo que', 'usted Necesito un otro correo', 'Pero okay', 'A qué correo', 'A qué correo perdón', 'Ah okay', 'Permíteme tantito le voy a pasar aquí a la secretaría para que le siga entendiendo porque yo andaría lleva perdido la idea', 'Deja le voy a pasar aquí voy a estar yo que la galería del teléfono', 'va a ser en en en página confío punto mx', 'porque tenemos una cosita', 'porque tenemos una cuenta', 'Tenemos una cuenta este usuario', 'de Dónde te

['la gripa', 'bueno', 'disculpe Es que siempre andamos acá', 'Baja California hermosa', 'átomos están hablando para querer', 'miren como rentas pues', 'según ellos para cuidarnos y no sé qué entonces', 'Aquí tengo aquí', 'a todos diciéndole si preguntan de dónde me habla ni para que eres y todo pero bueno', 'dígame', 'Pues ojalá que sea antes de lonche porque si no va a pasar lo mismo que el mes pasado Que se va a juntar el pago', 'muy amable Muchísimas gracias', 'buen día', 'igualmente Bye']


['la gripa', 'bueno', 'disculpe Es que siempre andamos acá', 'Baja California hermosa', 'átomos están hablando para querer', 'miren como rentas pues', 'según ellos para cuidarnos y no sé qué entonces', 'Aquí tengo aquí', 'a todos diciéndole si preguntan de dónde me habla ni para que eres y todo pero bueno', 'dígame', 'Pues ojalá que sea antes de lonche porque si no va a pasar lo mismo que el mes pasado Que se va a juntar el pago', 'muy amable Muchísimas gracias', 'buen día', 'igualmente Bye']


['Hola que tal Buenos días', 'Fabiola Barbosa', 'gracias gracias', 'Sí lo que pasa es que tenemos un crédito empresarial con ustedes', 'y esto queremos realizar el pago', 'pero no sabemos exactamente cuánto se tiene que depositar y éste el mont', 'Adónde hora que cuenta', 'los datos bancarios no los tenemos', 'Carlos tamez', 'no sé que ocupar', 'Antonio tamez', 'imagino que tal vez del suyo en algún punto tame', 'glacio galta', 'lacio', 'carta.com', 'Okay el de ahí me podrías indicar ya nada más vas fechas pero no el mundo', 'Bueno si por favor éste igual y no sé si quieras mandárselo a su correo', 'este o si me lo puedes mandar a mí y lo sé cómo tú lo veas o cómo se pueda no', 'Es que yo aquí también lo tengo a un lado de mi vecino', 'comunicar trotuman', 'Cómo saber cómo', 'copia de mi identificación', 'al mismo', 'MX', 'info@konfio.mx okay', 'muy bien perfecto entonces Quedó en espera de del calendario y los datos bancarios este y ya para que también se pueda autorizar a mi selecció

['Hola señorita buenas', 'bretón', 'Quiero ver cuánto es lo que tengo que pagar', 'tengo que depositar para este corte', 'no lo escuché señorita', 'Carmelo Cerro del otro', 'C de casa', 'r de ratón b de burro', 'r de ratón', 'bebé elefante', 'arroba outlook.es', 'este Carmen bretón Lugo', '13 de noviembre de 1983', 'señorita Gracias', 'Joan hart', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'señorita', 'este', 'por tres meses señorita', 'es que los pasados puse de a $6000', 'puse lo que era de este', 'señorita quiero estar', 'Ian holm', 'Joan hart', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'las 8', 'y los dos meses pasados de pagar eso 6539', 'señorita', 'Joan hart', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'porqu

...

['cambiar el monto a cambiar la mensualidad', 'bueno el dinero ya está en la cuenta nomás Espera que lo retiren entonces', 'espero', 'sería todo muy amable', 'gracias']


['Christian Madrid', 'es como la quinta vez que hablo', 'por chat', 'dos veces lo intenté', 'resulta que hace un mes', 'me ofrecieron un como estructura del crédito', 'donde me bajaba', 'el pago mensual', 'me aumentaron los meses', 'y además me van hacer un depósito como de $25000', 'pero resulta que se atravesó mi fecha de pago', 'Entonces como yo no miré yo pagué', 'no miré ningún depósito el día jueves', 'que me habían dicho', 'ya hace como 4 semanas o 5', 'y me dijeron No es que se atravesó el pago y que pues tienen que volver a hacer otro cálculo', 'porque con ese pago pues ya va a bajar y que no sé', 'pues ya es la quinta llamada', 'y nada más me dices lo vamos a ver', 'y pues no lo ven ni me hablan y me soluciona nada', 'slick como licenciados Lic punto Madrid arroba gmail.com', 'hoy Déjame entró', 'un segundito', 'Ingresar a mi cuenta', 'no me lo sé de memoria', 'Lázaro me apareció en esquina superior derecha', 'es el 35 610', 'Cristian Aurelia lamadrid monárrez', 'm a m 76 032

['Buenas tardes señor', 'este con Carmen bretón', 'señorita Quisiera saber cuánto es lo que voy a pagar ahorita', 'ver', 'burro', 'Aires', '1083', 'primero', 'es 83113', 'x8', 'cerbelu', 'arroba outlook.es', '772 153', '2303', 'señorita', 'fue', 'encomio', 'veintitantos', 'es que les depósito de aceite $1000 señorita', '1536 aeronca depositar', 'ya se los tengo que depositar hoy o señorita', 'Okay señorita Entonces cuando ya empezaría pagar bien así mis pagos ya completos', 'En cuántos meses', 'es que me mandaron una tabla de amortización', 'Ponme inquietante pagar 17155 y es lo que me hice bolas que no entiendo cómo voy a pagar ya O sea ya cuando ya va hacer el pago', 'incluyendo el capital con el interés', 'Sí señorita', 'otra vez señorita por favor', 'Qué pasa si está diciéndole', 'noviembre', 'Face', '10618.40', 'Okay para marcarle', 'señorita por favor', 'roomba', 'Okay señorita', 'otra pregunta qué datos tengo que poner para el pago', 'red', 'red amigos', 'el concepto', 'el conce

['grabe su mensaje despues del tono']


['y buenas tardes', 'Eugenio Sánchez Salas', 'vivimos aquí en la plataforma', 'un ofrecimiento de', 'de liquidar la deuda de una sola exhibición y quería ver este pues más que nada', 'a quedar unos puntitos ahí no sé si si con usted sea la persona', 'Sánchez Salas', 'el 7 de noviembre del 70', 'no es una es como en la plataforma The Killer', 'desconfío personal ahí nos nos que nos enviaron una', 'cómo queda una promoción', 'una promoción especial', 'para', 'planes de pago', 'suena bien pero quería quería quería', 'era confirmar', 'es correcto', 'la primera', 'la primera es que no sólo exhibición', 'me pregunta mi pregunta es', 'al pagar esto no afecta algún problema con buró de crédito o alguna situación de esas que', 'que posteriormente nosotros vayamos a ocupar y nos afecte por por hacer ese tipo de', 'el', 'The Kid', 'Ah Okay qué', 'sí', 'Aquí está chateando estamos aquí lo que quita con el asesor y aquí en la oficina porque tenemos el pendiente intercede estamos viendo', 'ver la op

In [27]:
def plot_sentiment_binary(sample,sent_col='sentiment_binary'):
  negativo=np.array(sample[sent_col]['negativo'])
  positivo=np.array(sample[sent_col]['positivo'])
  diff=positivo-negativo
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=list(range(len(diff))), y=diff,name='sentiment')) # fill down to xaxi
  fig.show()


In [15]:
for sample in samples.to_dict('records'):
  print(sample['transcript'])
  plot_sentiment_binary(sample)

['Sí buenas tardes', 'habló a Eugenio', 'Mira por aquí me interesa estoy interesado en una reestructuración', 'multiservicios punto azul o arroba hotmail.com', 'Eugenio Sánchez Salas', 'el 7 de noviembre del 70 fecha de nacimiento el 7 de noviembre 70', 'correcto', 'Sí está bien', 'Bueno si estoy bien ya estamos ahí', 'Ian holm', 'Joan hart', 'John Holt', 'John Holt', 'arco el correo multiservicios multi el mismo correo multiservicios punto solo', 'un correo que lo tengo que compartir', 'Perdón no no le no le no le no le entiendo O sea qué es lo que', 'usted Necesito un otro correo', 'Pero okay', 'A qué correo', 'A qué correo perdón', 'Ah okay', 'Permíteme tantito le voy a pasar aquí a la secretaría para que le siga entendiendo porque yo andaría lleva perdido la idea', 'Deja le voy a pasar aquí voy a estar yo que la galería del teléfono', 'va a ser en en en página confío punto mx', 'porque tenemos una cosita', 'porque tenemos una cuenta', 'Tenemos una cuenta este usuario', 'de Dónde te

['la gripa', 'bueno', 'disculpe Es que siempre andamos acá', 'Baja California hermosa', 'átomos están hablando para querer', 'miren como rentas pues', 'según ellos para cuidarnos y no sé qué entonces', 'Aquí tengo aquí', 'a todos diciéndole si preguntan de dónde me habla ni para que eres y todo pero bueno', 'dígame', 'Pues ojalá que sea antes de lonche porque si no va a pasar lo mismo que el mes pasado Que se va a juntar el pago', 'muy amable Muchísimas gracias', 'buen día', 'igualmente Bye']


['la gripa', 'bueno', 'disculpe Es que siempre andamos acá', 'Baja California hermosa', 'átomos están hablando para querer', 'miren como rentas pues', 'según ellos para cuidarnos y no sé qué entonces', 'Aquí tengo aquí', 'a todos diciéndole si preguntan de dónde me habla ni para que eres y todo pero bueno', 'dígame', 'Pues ojalá que sea antes de lonche porque si no va a pasar lo mismo que el mes pasado Que se va a juntar el pago', 'muy amable Muchísimas gracias', 'buen día', 'igualmente Bye']


['Hola que tal Buenos días', 'Fabiola Barbosa', 'gracias gracias', 'Sí lo que pasa es que tenemos un crédito empresarial con ustedes', 'y esto queremos realizar el pago', 'pero no sabemos exactamente cuánto se tiene que depositar y éste el mont', 'Adónde hora que cuenta', 'los datos bancarios no los tenemos', 'Carlos tamez', 'no sé que ocupar', 'Antonio tamez', 'imagino que tal vez del suyo en algún punto tame', 'glacio galta', 'lacio', 'carta.com', 'Okay el de ahí me podrías indicar ya nada más vas fechas pero no el mundo', 'Bueno si por favor éste igual y no sé si quieras mandárselo a su correo', 'este o si me lo puedes mandar a mí y lo sé cómo tú lo veas o cómo se pueda no', 'Es que yo aquí también lo tengo a un lado de mi vecino', 'comunicar trotuman', 'Cómo saber cómo', 'copia de mi identificación', 'al mismo', 'MX', 'info@konfio.mx okay', 'muy bien perfecto entonces Quedó en espera de del calendario y los datos bancarios este y ya para que también se pueda autorizar a mi selecció

['Hola señorita buenas', 'bretón', 'Quiero ver cuánto es lo que tengo que pagar', 'tengo que depositar para este corte', 'no lo escuché señorita', 'Carmelo Cerro del otro', 'C de casa', 'r de ratón b de burro', 'r de ratón', 'bebé elefante', 'arroba outlook.es', 'este Carmen bretón Lugo', '13 de noviembre de 1983', 'señorita Gracias', 'Joan hart', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'señorita', 'este', 'por tres meses señorita', 'es que los pasados puse de a $6000', 'puse lo que era de este', 'señorita quiero estar', 'Ian holm', 'Joan hart', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'las 8', 'y los dos meses pasados de pagar eso 6539', 'señorita', 'Joan hart', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'John Holt', 'porqu

...

['cambiar el monto a cambiar la mensualidad', 'bueno el dinero ya está en la cuenta nomás Espera que lo retiren entonces', 'espero', 'sería todo muy amable', 'gracias']


['Christian Madrid', 'es como la quinta vez que hablo', 'por chat', 'dos veces lo intenté', 'resulta que hace un mes', 'me ofrecieron un como estructura del crédito', 'donde me bajaba', 'el pago mensual', 'me aumentaron los meses', 'y además me van hacer un depósito como de $25000', 'pero resulta que se atravesó mi fecha de pago', 'Entonces como yo no miré yo pagué', 'no miré ningún depósito el día jueves', 'que me habían dicho', 'ya hace como 4 semanas o 5', 'y me dijeron No es que se atravesó el pago y que pues tienen que volver a hacer otro cálculo', 'porque con ese pago pues ya va a bajar y que no sé', 'pues ya es la quinta llamada', 'y nada más me dices lo vamos a ver', 'y pues no lo ven ni me hablan y me soluciona nada', 'slick como licenciados Lic punto Madrid arroba gmail.com', 'hoy Déjame entró', 'un segundito', 'Ingresar a mi cuenta', 'no me lo sé de memoria', 'Lázaro me apareció en esquina superior derecha', 'es el 35 610', 'Cristian Aurelia lamadrid monárrez', 'm a m 76 032

['Buenas tardes señor', 'este con Carmen bretón', 'señorita Quisiera saber cuánto es lo que voy a pagar ahorita', 'ver', 'burro', 'Aires', '1083', 'primero', 'es 83113', 'x8', 'cerbelu', 'arroba outlook.es', '772 153', '2303', 'señorita', 'fue', 'encomio', 'veintitantos', 'es que les depósito de aceite $1000 señorita', '1536 aeronca depositar', 'ya se los tengo que depositar hoy o señorita', 'Okay señorita Entonces cuando ya empezaría pagar bien así mis pagos ya completos', 'En cuántos meses', 'es que me mandaron una tabla de amortización', 'Ponme inquietante pagar 17155 y es lo que me hice bolas que no entiendo cómo voy a pagar ya O sea ya cuando ya va hacer el pago', 'incluyendo el capital con el interés', 'Sí señorita', 'otra vez señorita por favor', 'Qué pasa si está diciéndole', 'noviembre', 'Face', '10618.40', 'Okay para marcarle', 'señorita por favor', 'roomba', 'Okay señorita', 'otra pregunta qué datos tengo que poner para el pago', 'red', 'red amigos', 'el concepto', 'el conce

['grabe su mensaje despues del tono']


['y buenas tardes', 'Eugenio Sánchez Salas', 'vivimos aquí en la plataforma', 'un ofrecimiento de', 'de liquidar la deuda de una sola exhibición y quería ver este pues más que nada', 'a quedar unos puntitos ahí no sé si si con usted sea la persona', 'Sánchez Salas', 'el 7 de noviembre del 70', 'no es una es como en la plataforma The Killer', 'desconfío personal ahí nos nos que nos enviaron una', 'cómo queda una promoción', 'una promoción especial', 'para', 'planes de pago', 'suena bien pero quería quería quería', 'era confirmar', 'es correcto', 'la primera', 'la primera es que no sólo exhibición', 'me pregunta mi pregunta es', 'al pagar esto no afecta algún problema con buró de crédito o alguna situación de esas que', 'que posteriormente nosotros vayamos a ocupar y nos afecte por por hacer ese tipo de', 'el', 'The Kid', 'Ah Okay qué', 'sí', 'Aquí está chateando estamos aquí lo que quita con el asesor y aquí en la oficina porque tenemos el pendiente intercede estamos viendo', 'ver la op

In [16]:
predict_binary=partial(predict,labels=['positivo','negativo'],multi_class=True)
transcripts=interactions_calls['client_sent'].values 
with Pool(8) as p:
  interactions_calls['sentiment_transcribe']=[sentiments for sentiments in tqdm(p.imap(predict,transcripts),total=len(transcripts))]
with Pool(8) as p:
  interactions_calls['sentiment_binary_transcribe']=[sentiments for sentiments in tqdm(p.imap(predict_binary,transcripts),total=len(transcripts))]

In [19]:
samples=interactions_calls[interactions_calls.sentiment.notnull()][['client_sent','sentiment_transcribe','sentiment_binary_transcribe']]

In [21]:
for sample in samples.to_dict('records'):
  print(sample['client_sent'])
  plot_lines(sample['sentiment_transcribe'])
  plot_lines(sample['sentiment_binary_transcribe'])

['Sí , Buenas tardes . Este hablo un genio . Mire por aquí . Me estoy interesado en una restructuración de mi crédito . Eh ? Multiservicios punto A solo arroba hotmail punto com Mmm . Sí , sí . Eugenio Sánchez Salas , el siete de legumbre , el setenta , fecha de nacimiento El siete de noviembre setenta . Sí , correcto . Sí , está bien . Sí , mmm . Claro , Okay . Sí , Bueno , bueno , si esta bien . Ya estamos ahí . Sí , y un honor y uno North y uno North y uno no . Sí , sí , está bien . Un qué ? Perdón ? A que el correo El multiservicios multi .', 'El mismo correo multiservicios , punto a Solo un correo que lo tengo que compartir a ver a verdad ? No , no le no le no le no le entiendo .', 'Osea que es lo que yo usted necesita un con otro correo . Ah , okay . Se le manda . Sí , está bien . Okay , pero okay . Sí , para que el correo aquí el correo . Perdón , Info . Ah , okay . A ver , permítame tantito Le voy a pasar aquí a la secretaria para que le sigue entendiendo ?', 'Porque yo en real

['así sería la gripe . Buenas tardes . Quién la busca ? Perdón ? De dónde ? Perdón . Confío , Sí . Ah , ok . Permítame tantito . Bueno , cómo ? Disculpe , Es que siempre andamos a la CAM . Vas a California ?', 'Nos cada rato nos están hablando para querer cómo viven como rentas .', 'Pues según ellos , para cuidarnos y no sé qué . Entonces aquí tengo aquí Ah , a todos diciéndoles pregunten de dónde me hablan y para qué es y todo .', 'Pero bueno , dígame . Ajá . Okay .', 'Pues ojalá que sea antes del once , porque si no me va a pasar lo mismo que el mes pasado , que se va a juntar el pago . Okay , muy amable .', 'Muchísimas gracias , Bai . Buen día . Igualmente , Bay .']


['así sería la gripe . Buenas tardes . Quién la busca ? Perdón ? De dónde ? Perdón . Confío , Sí . Ah , ok . Permítame tantito . Bueno , cómo ? Disculpe , Es que siempre andamos a la CAM . Vas a California ?', 'Nos cada rato nos están hablando para querer cómo viven como rentas .', 'Pues según ellos , para cuidarnos y no sé qué . Entonces aquí tengo aquí Ah , a todos diciéndoles pregunten de dónde me hablan y para qué es y todo .', 'Pero bueno , dígame . Ajá . Okay .', 'Pues ojalá que sea antes del once , porque si no me va a pasar lo mismo que el mes pasado , que se va a juntar el pago . Okay , muy amable .', 'Muchísimas gracias , Bai . Buen día . Igualmente , Bay .']


['Hola , Que tal ? Buenos días . Como todavía no me gusta . Gracias . Gracias . Eh , sí , lo que pasa es que tenemos un un crédito empresa de ver con ustedes y esto podemos realizar el pago , pero no sabemos exactamente cuanto se tiene que depositar .', 'Y este el monte , eh ? A dónde osea que cuenta los datos bancarios No los tenemos . No , en mi jefe Carlos también . No sé quién ocupa Carlos Antonio Thames bye . Sí , es correcto . Ah , me imagino que tal vez el suyo es Carlos .', 'Punto también arroba C Lazio C alta es G lacio . Carta punto com . Okay . Este ahí . Me podrías ? Ahí me indicaría nada más las fechas , pero no el mundo .', 'Ah , bueno , sí , por favor , este igual .', 'Y no sé si quieras mandar celular su correo este o si me lo puedes mandar a mi . No sé , como tú lo veas o como como se pueda , no este yo aquí también lo tengo .', 'Está a un lado , es muchísimo bueno , huevo como un cartero .', 'Tú me tomé , lo podemos saber como mejor . Okay , gracias .', 'Copia de mi i

['Pues los señorita buenas con terminan Bretón . Sí , tres , dos . Y usted quisiera ver cuanto es lo que tengo que pagar . Que tengo que depositar para este corte . Sí . Oh , qué señorita , No la escucho bien , señorita , Este cargo de lujo Cerro del otro ce de casa a erre de ratón Be de burro , erre de ratón de de elefantes . Ele de Lalo . Uh , arroba o cloud punto es este ? Carmen Bretón Lugo .', 'Trece de noviembre de mil novecientos ochenta y tres . Si , señorita . Gracias Y un honor y uno North y uno no . Y uno No . Y un honor y uno no . Sí , señor , sí , pero también señor . Seis . Sí , este , eh , por tres veces , señorita Este .', 'En los pasados puse de seis mil pesos . Pues es lo que era de este intereses . Si , señorita .', 'Que las y un honor y un honor y uno no y un honor y un honor y uno No y uno North y uno North y uno North y uno North Y un honor y uno North y uno North y uno North y un honor y uno no y uno no y uno North y uno North y un honor y uno North y uno North .

...

['Bueno , okay . Dígame . Sí , sí , sí . Ajá .', 'Porque entonces a cambiar el monto y va a cambiar la mensualidad . Ah , okay . Bueno , el dinero ya está en la cuenta nomás . Espera que lo retiren entonces . Está bien . Ah , ok . Muy bien , entonces , espero No , señorita , sería todo muy amable .', 'Igualmente . Gracias .']


['con Cristian Madrid . Mire , señorita , es como la quinta vez que hablo y por chat dos veces lo intenté .', 'Resulta que hace un mes me ofrecieron un cómo reestructura del crédito , donde me bajaba el pago mensual , me comentaban , los mueve los meses y además me van a hacer un depósito como de veinticinco mil pesos .', 'Pero resulta que se atravesó mi fecha de pago . Entonces , como yo no , mire , yo pague .', 'Pero yo no miré ningún deposito el día jueves que me habían dicho ya hace como cuatro semanas o cinco .', 'Pues hable y me dijeron no , es que se atravesó el pago y que pues tienen que volver a hacer otro cálculo , porque cuando ese pago pues ya va a bajar y que no sé qué okay , pues ya es la quinta llamada y nada más me dice No , vamos a ver y pues no lo ven ni me hablan y me solucionan . Nada', 'Es like como licenciado li punto Madrid arroba gmail punto com . Hoy déjame , entró , Deme un segundito .', 'Qué ingresa nueva a mi cuenta ?', 'Porque no me lo sé .', 'De memoria , 

['Buenas tardes , señor . Este con Carmen Bretón . Carmen Bretón . Hola , señorita . Quisiera saber cuánto es lo que voy a pagar ahorita . Okay . Be de burro e ele ge hay Es tu Bill ? Ochenta once . Trece , ochenta y once . Va primero el mes . El año es ochenta y tres once tres Equis ocho Cardelús , arroba , aclu punto es este .', 'Setecientos setenta y dos , cientos cincuenta y tres , doce cero tres . Qué señalizaciones fue así ? No me acuerdo como en en junio .', 'Veinti tantos . Pues que les les deposito de a seis mil pesos , señor .', 'Seis mil quinientos ochenta y seis de los Tengo que depositar . Agradecimiento . Ya se los tengo que depositar los dos , señorita . Pues okay , señorita . Entonces , cuando ya empezaría pagar bien así mis pagos ya completo . Ajá . Me En cuántos meses ? Es que me mandaron una tarde demo , tiza ción y me ponen que prepotente pagar diecisiete mil , ciento cincuenta y cinco Y es lo que me hice bolas . Yo no entiendo cómo voy a pagar ya . O sea , ya cuand

['grabe su mensaje después del tono .']


['y buenas tardes , eh , Eugenio Sánchez Salas . Sí , mire , vivimos aquí en la plataforma , un ofrecimiento de del equidad , la deuda en una sola exhibición .', 'Y quería ver este Pues más que nada , a quedar unos puntitos ahí , no sé si si con usted sea la persona Sánchez Salas el siete de noviembre setenta . Sí , no , no es una es como en la plataforma de aquí , de de eh , de confío personal .', 'Ahí nos nos este Nos enviaron una como que una promoción , Una promoción especial , eh ? Para , claro que pago relación y y suena bien , pero quería , quería sería Mira , quería confirmar Sí , sí , correcto . Sí , es correcto . Sí , la primera , la primera es tiene sola exhibición . Sí , eh , Mi pregunta , Mi pregunta es eh , eh , al pagar esto no afecta algún problema con buró de crédito o alguna situación de esas que que posteriormente nosotros vayamos ocupar y nos afecte por puede hacer ese tipo de El de quita , Sí , Ah , ok . Sí , sí , es correcto . Sí , eh Sí , sí . Aquí está , chequea

In [28]:
for sample in samples.to_dict('records'):
  print(sample['client_sent'])
  plot_sentiment_binary(sample,sent_col='sentiment_binary_transcribe')

['Sí , Buenas tardes . Este hablo un genio . Mire por aquí . Me estoy interesado en una restructuración de mi crédito . Eh ? Multiservicios punto A solo arroba hotmail punto com Mmm . Sí , sí . Eugenio Sánchez Salas , el siete de legumbre , el setenta , fecha de nacimiento El siete de noviembre setenta . Sí , correcto . Sí , está bien . Sí , mmm . Claro , Okay . Sí , Bueno , bueno , si esta bien . Ya estamos ahí . Sí , y un honor y uno North y uno North y uno no . Sí , sí , está bien . Un qué ? Perdón ? A que el correo El multiservicios multi .', 'El mismo correo multiservicios , punto a Solo un correo que lo tengo que compartir a ver a verdad ? No , no le no le no le no le entiendo .', 'Osea que es lo que yo usted necesita un con otro correo . Ah , okay . Se le manda . Sí , está bien . Okay , pero okay . Sí , para que el correo aquí el correo . Perdón , Info . Ah , okay . A ver , permítame tantito Le voy a pasar aquí a la secretaria para que le sigue entendiendo ?', 'Porque yo en real

['así sería la gripe . Buenas tardes . Quién la busca ? Perdón ? De dónde ? Perdón . Confío , Sí . Ah , ok . Permítame tantito . Bueno , cómo ? Disculpe , Es que siempre andamos a la CAM . Vas a California ?', 'Nos cada rato nos están hablando para querer cómo viven como rentas .', 'Pues según ellos , para cuidarnos y no sé qué . Entonces aquí tengo aquí Ah , a todos diciéndoles pregunten de dónde me hablan y para qué es y todo .', 'Pero bueno , dígame . Ajá . Okay .', 'Pues ojalá que sea antes del once , porque si no me va a pasar lo mismo que el mes pasado , que se va a juntar el pago . Okay , muy amable .', 'Muchísimas gracias , Bai . Buen día . Igualmente , Bay .']


['así sería la gripe . Buenas tardes . Quién la busca ? Perdón ? De dónde ? Perdón . Confío , Sí . Ah , ok . Permítame tantito . Bueno , cómo ? Disculpe , Es que siempre andamos a la CAM . Vas a California ?', 'Nos cada rato nos están hablando para querer cómo viven como rentas .', 'Pues según ellos , para cuidarnos y no sé qué . Entonces aquí tengo aquí Ah , a todos diciéndoles pregunten de dónde me hablan y para qué es y todo .', 'Pero bueno , dígame . Ajá . Okay .', 'Pues ojalá que sea antes del once , porque si no me va a pasar lo mismo que el mes pasado , que se va a juntar el pago . Okay , muy amable .', 'Muchísimas gracias , Bai . Buen día . Igualmente , Bay .']


['Hola , Que tal ? Buenos días . Como todavía no me gusta . Gracias . Gracias . Eh , sí , lo que pasa es que tenemos un un crédito empresa de ver con ustedes y esto podemos realizar el pago , pero no sabemos exactamente cuanto se tiene que depositar .', 'Y este el monte , eh ? A dónde osea que cuenta los datos bancarios No los tenemos . No , en mi jefe Carlos también . No sé quién ocupa Carlos Antonio Thames bye . Sí , es correcto . Ah , me imagino que tal vez el suyo es Carlos .', 'Punto también arroba C Lazio C alta es G lacio . Carta punto com . Okay . Este ahí . Me podrías ? Ahí me indicaría nada más las fechas , pero no el mundo .', 'Ah , bueno , sí , por favor , este igual .', 'Y no sé si quieras mandar celular su correo este o si me lo puedes mandar a mi . No sé , como tú lo veas o como como se pueda , no este yo aquí también lo tengo .', 'Está a un lado , es muchísimo bueno , huevo como un cartero .', 'Tú me tomé , lo podemos saber como mejor . Okay , gracias .', 'Copia de mi i

['Pues los señorita buenas con terminan Bretón . Sí , tres , dos . Y usted quisiera ver cuanto es lo que tengo que pagar . Que tengo que depositar para este corte . Sí . Oh , qué señorita , No la escucho bien , señorita , Este cargo de lujo Cerro del otro ce de casa a erre de ratón Be de burro , erre de ratón de de elefantes . Ele de Lalo . Uh , arroba o cloud punto es este ? Carmen Bretón Lugo .', 'Trece de noviembre de mil novecientos ochenta y tres . Si , señorita . Gracias Y un honor y uno North y uno no . Y uno No . Y un honor y uno no . Sí , señor , sí , pero también señor . Seis . Sí , este , eh , por tres veces , señorita Este .', 'En los pasados puse de seis mil pesos . Pues es lo que era de este intereses . Si , señorita .', 'Que las y un honor y un honor y uno no y un honor y un honor y uno No y uno North y uno North y uno North y uno North Y un honor y uno North y uno North y uno North y un honor y uno no y uno no y uno North y uno North y un honor y uno North y uno North .

...

['Bueno , okay . Dígame . Sí , sí , sí . Ajá .', 'Porque entonces a cambiar el monto y va a cambiar la mensualidad . Ah , okay . Bueno , el dinero ya está en la cuenta nomás . Espera que lo retiren entonces . Está bien . Ah , ok . Muy bien , entonces , espero No , señorita , sería todo muy amable .', 'Igualmente . Gracias .']


['con Cristian Madrid . Mire , señorita , es como la quinta vez que hablo y por chat dos veces lo intenté .', 'Resulta que hace un mes me ofrecieron un cómo reestructura del crédito , donde me bajaba el pago mensual , me comentaban , los mueve los meses y además me van a hacer un depósito como de veinticinco mil pesos .', 'Pero resulta que se atravesó mi fecha de pago . Entonces , como yo no , mire , yo pague .', 'Pero yo no miré ningún deposito el día jueves que me habían dicho ya hace como cuatro semanas o cinco .', 'Pues hable y me dijeron no , es que se atravesó el pago y que pues tienen que volver a hacer otro cálculo , porque cuando ese pago pues ya va a bajar y que no sé qué okay , pues ya es la quinta llamada y nada más me dice No , vamos a ver y pues no lo ven ni me hablan y me solucionan . Nada', 'Es like como licenciado li punto Madrid arroba gmail punto com . Hoy déjame , entró , Deme un segundito .', 'Qué ingresa nueva a mi cuenta ?', 'Porque no me lo sé .', 'De memoria , 

['Buenas tardes , señor . Este con Carmen Bretón . Carmen Bretón . Hola , señorita . Quisiera saber cuánto es lo que voy a pagar ahorita . Okay . Be de burro e ele ge hay Es tu Bill ? Ochenta once . Trece , ochenta y once . Va primero el mes . El año es ochenta y tres once tres Equis ocho Cardelús , arroba , aclu punto es este .', 'Setecientos setenta y dos , cientos cincuenta y tres , doce cero tres . Qué señalizaciones fue así ? No me acuerdo como en en junio .', 'Veinti tantos . Pues que les les deposito de a seis mil pesos , señor .', 'Seis mil quinientos ochenta y seis de los Tengo que depositar . Agradecimiento . Ya se los tengo que depositar los dos , señorita . Pues okay , señorita . Entonces , cuando ya empezaría pagar bien así mis pagos ya completo . Ajá . Me En cuántos meses ? Es que me mandaron una tarde demo , tiza ción y me ponen que prepotente pagar diecisiete mil , ciento cincuenta y cinco Y es lo que me hice bolas . Yo no entiendo cómo voy a pagar ya . O sea , ya cuand

['grabe su mensaje después del tono .']


['y buenas tardes , eh , Eugenio Sánchez Salas . Sí , mire , vivimos aquí en la plataforma , un ofrecimiento de del equidad , la deuda en una sola exhibición .', 'Y quería ver este Pues más que nada , a quedar unos puntitos ahí , no sé si si con usted sea la persona Sánchez Salas el siete de noviembre setenta . Sí , no , no es una es como en la plataforma de aquí , de de eh , de confío personal .', 'Ahí nos nos este Nos enviaron una como que una promoción , Una promoción especial , eh ? Para , claro que pago relación y y suena bien , pero quería , quería sería Mira , quería confirmar Sí , sí , correcto . Sí , es correcto . Sí , la primera , la primera es tiene sola exhibición . Sí , eh , Mi pregunta , Mi pregunta es eh , eh , al pagar esto no afecta algún problema con buró de crédito o alguna situación de esas que que posteriormente nosotros vayamos ocupar y nos afecte por puede hacer ese tipo de El de quita , Sí , Ah , ok . Sí , sí , es correcto . Sí , eh Sí , sí . Aquí está , chequea